<a href="https://colab.research.google.com/github/sonialouise/ts_class/blob/main/notebooks/SpontaneousActivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spontaneous Neuronal Activity Classifier
This notebook allows users to run a pretrained model on their own data to identify neurons exhibiting spontaneous activity. Furthermore, users can fine-tune this model using their own data.

In [1]:
#@markdown The Github repository with all custom functions, the model training datasets and the pretrained models are copied to the environment of this notebook. This will take a couple of seconds.

#@markdown *Note: You can check the code underlying each cell by double-clicking on it.*

import os

# If in Colab and not yet downloaded, download GitHub repository and change working directory
if os.getcwd() == '/content':  
    !git clone https://github.com/sonialouise/ts_class.git
    os.chdir('ts_class')
    
# If executed as jupyter notebook on own computer, change to parent directory for imports
if os.path.basename( os.getcwd() ) == 'activity_classifier':
    %cd ..
    print('New working directory:', os.getcwd() )

fatal: destination path 'ts_class' already exists and is not an empty directory.


In [2]:
!git pull 

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/sonialouise/ts_class
   6514133..69ebaed  main       -> origin/main
Updating 6514133..69ebaed
Fast-forward
 activity_classifier/prepare_data.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [3]:
#@markdown Download packages required to run the model

%%capture
!pip install -r requirements.txt

# python imports
import pickle
import argparse
import logging
import pandas as pd

# ts_class packages, imported from the downloaded Github repository
from activity_classifier.main import run_model
from activity_classifier.config import TSF_MODEL, RISE_MODEL, OBS, PREDICTION, OUTPUT_PATH
from activity_classifier.prepare_data import prepare_data
from activity_classifier.retrain_models import retrain_tsf, retrain_rise

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [4]:
#@markdown If you are testing the script, leave the below two fields unchanged. If you want to apply the model to your own data, you need to upload your data first. 

#@markdown To upload data, click on the **folder symbol ("Files")** on the left side of the Colaboratory notebook, and select 'Upload to session storage'. Next, indicate the file path of the uploaded file in the variable **`file_path`** below. Finally, indicate the total duration in seconds (decimal) of your recordings in the variable **`duration`**, and the required sampling rate (frames per second) in the variable **`sampling_rate`**..

#@markdown Note that for data longer than 550 frames, only the first 550 frames will be used for the prediction, unless the model is retrained with the higher frame count.

file_path = "data/test_set.csv" #@param {type:"string"}

duration = 137.5 #@param {type:"number"}  

sampling_rate = 4 #@param {type: "number"}

In [5]:
#@markdown Play this cell to retrain the model. If this is not required, skip this cell.<br />
#@markdown Note that training data should contain the following:<br />
#@markdown - Rows containing individual cell recordings, with columns indicating timepoints
#@markdown - Numbered column headers (starting at zero) to indicate the captured timepoints
#@markdown - A 'status' column to the right of recordings, containing 'active' or 'inactive' labels for each row <br />

#@markdown Ensure that data file path and frame count have been updated in the above cell.
    
logging.info("1. Reading csv file...")
data = pd.read_csv('data/training_data.csv', header=0)
logging.info("2. Normalising data...")
total_frames = int(duration * sampling_rate)
trace_data = data.iloc[:, 0:total_frames]
data = pd.concat([prepare_data(trace_data, duration, sampling_rate), data['status']], axis=1)
logging.info("3. Retraining Time Series Classifier...")
retrain_tsf(data)
logging.info("4. Retraining Random Interval Spectral Ensemble...")
retrain_rise(data)
logging.info("5. Retraining Complete")

RESULTS TSF: {'ACCURACY': 0.9105495598256852, 'PRECISION': 0.8283819604233779, 'RECALL': 0.7232455632945328}
RESULTS RISE: {'ACCURACY': 0.9701792869468573, 'PRECISION': 0.9484897860759929, 'RECALL': 0.9023335231146226}


In [10]:
#@markdown Play this cell to run the model on your data. The model predictions can be found in the "Files" folder under ts_class -> data.

logging.info("1. Reading csv file...")
data = pd.read_csv(file_path, header=0)
logging.info("2. Normalising data...")
total_frames = int(duration * sampling_rate)
trace_data = data.iloc[:, 0:total_frames]
data = prepare_data(trace_data, duration, sampling_rate)
logging.info("3. Predicting with TimeSeries Forest Classifier...")
data = run_model(data, TSF_MODEL, 'TSF')
logging.info("4. Predicting with Random Interval Spectral Ensemble...")
data = run_model(data, RISE_MODEL, 'RISE')
logging.info("5. Saving output...")
data.to_csv(OUTPUT_PATH)
logging.info("6. Process complete")

/content/ts_class/activity_classifier/prepare_data.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[OBS] = [interpolate_data(row, seconds, end_frame_rate) for row in np.array(data)]
